<a href="https://colab.research.google.com/github/Freemanj19/Kepler-62f/blob/main/Kepler62f_obliq_evol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rebound

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
from scipy.integrate import odeint
import rebound
import threading
import os

#Laskar paper: https://physics.ucf.edu/~britt/Geophysics/Readings/G15-Evolution%20of%20obliquity.pdf
#Quarles paper: https://ui.adsabs.harvard.edu/abs/2022MNRAS.509.2736Q/abstract
#Quarles paper: https://ui.adsabs.harvard.edu/abs/2020AsBio..20...73Q/abstract

lock = threading.Lock()
home = os.getcwd() + "/"
data_dir = home + "K62_orb/"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)


M_e = 3.0035e-6
Mstar_K62 = 0.69
K62b = [0.0552802,0.07071068,0.883,278.13010,90.373788,74.589661,2.71*M_e] #(semimajor axis, ecc, inclination, omega, Omega, Mean Anomaly, mass in M_sun)
K62c = [0.0928576,0.08602325,0.560,54.462322,90.473841,327.80383578,0.128*M_e]
K62d = [0.1195013,0.09486833,0.300,288.43494,90.002197,243.09602,4.94*M_e]
K62e = [0.4263096,0.13000000,0.303,292.61986,90.303058,63.220181,4.05*M_e]
K62f = [0.7176171,0.09433981,0.488,57.994616,90.478155,176.54639,3.57*M_e]

P_K62b = np.sqrt(K62b[0]**3/Mstar_K62)*365.25
print(P_K62b*0.05)

tscale = 5e5 #500,000 years for integration time
days = np.arange(0, (tscale+1)*365.25, 365.25)

pl_param = [K62b,K62c,K62d,K62e,K62f]
N_bodies = len(pl_param) + 1
for i in range(0,len(pl_param)):
    temp = pl_param[i]
    temp[2:6] = np.radians(temp[2:6])
    pl_param[i] = temp


def write2file(f_str,head,output_fldr,fn):
    lock.acquire() # thread blocks at this line until it can obtain lock
    if head == "head":
        f = open(output_fldr+fn,'w')
    else:
        f = open(output_fldr+fn,'a')
    f.write(f_str)
    f.close()
    lock.release()

def X2eps(x):
    return np.degrees(np.arccos(x))

def unfold_2pi(t,x):
    x_unfold = np.zeros(len(t))
    NN = 0
    for j in range(0,len(t)-1):
        diff = x[j+1] - x[j]
        if diff > 1.:
            NN -= 1
        if diff < -np.pi:
            NN += 1
        x_unfold[j+1] = x[j+1] + 2.*NN*np.pi
    return x_unfold

def deriv(y,t,alphat,ecct,CAt,CBt,CCt):
    Xt, psit = y
    if np.abs(Xt) < 1:
        term = Xt/np.sqrt(1.-Xt**2)
        root = np.sqrt(1.-Xt**2)
    else:
        term = -1.
        root = 0.
        
    dHdpsi = root*(CAt*np.cos(psit) - CBt*np.sin(psit))
    dHdX = alphat*Xt/(1.-ecct**2)**1.5-term*(CAt*np.sin(psit)+CBt*np.cos(psit)) - 2.*CCt
    return [-dHdpsi,dHdX]

def run_obl(days,semi_t,ecc_t,inc_t,argP_t,RA_t,eps,alpha,dest):
    #semi_t,ecc_t,inc_t,argP_t,RA_t are arrays from run_sim()
    time_arr = np.copy(days)/365.25

    inc = np.radians(inc_t)
    RA = np.radians(RA_t)
    RA_cont = unfold_2pi(time_arr,RA)

    q = np.sin(inc/2.)*np.cos(RA)
    p = np.sin(inc/2.)*np.sin(RA)

    qt = UnivariateSpline(time_arr, q, s=0)
    pt = UnivariateSpline(time_arr, p, s=0)

    pdot = pt.derivative()(time_arr)
    qdot = qt.derivative()(time_arr)

    CC = qt(time_arr)*pdot-pt(time_arr)*qdot
    CA = 2./np.sqrt(1.-pt(time_arr)**2-qt(time_arr)**2)*(qdot + pt(time_arr)*CC)
    CB = 2./np.sqrt(1.-pt(time_arr)**2-qt(time_arr)**2)*(pdot - qt(time_arr)*CC)
    
    
    Chi,psi = np.zeros(len(time_arr)),np.zeros(len(time_arr))
    i_spin = np.zeros(len(time_arr))
    if eps == 0:
        eps = 1e-6
    Chi[0] = np.cos(np.radians(eps))
    psi[0] = 0 #const_psi
    
    write2file("%1.6e %1.6f %1.6f %3.5f %3.5f %3.5f %3.5f\n" % (0,semi_t[0],ecc_t[0],argP_t[0],RA_t[0],eps,np.degrees(psi[0])),'head',dest,'obl_data.txt')
    tstep = time_arr[1]/10.
    
    for t in range(1,len(time_arr)):
        delta_t = np.arange(time_arr[t-1],time_arr[t]+tstep,tstep)
        y0 = [Chi[t-1],psi[t-1]]
        alp = alpha/3600.*np.pi/180.
        temp = odeint(deriv,y0,delta_t,args=(alp,ecc_t[t-1],CA[t-1],CB[t-1],CC[t-1]),atol=1e-10,rtol=1e-10)
        Chi[t],psi[t] = temp[-1,0],temp[-1,1]
        if not np.isfinite(Chi[t]):
            Chi[t] = 1.

        ArgP = argP_t[t] % 360.
        LongP = RA_t[t] % 360.
        Obl = X2eps(Chi[t])
        PrecA = np.degrees(psi[t])%360.
        write2file("%1.6e %1.6f %1.6f %3.5f %3.5f %3.5f %3.5f\n" % (time_arr[t],semi_t[t],ecc_t[t],ArgP,LongP,Obl,PrecA),'foot',dest,'obl_data.txt')

    return

def run_sim():
    sim = rebound.Simulation()
    sim.integrator = "whfast"
    sim.units = ('days', 'AU', 'Msun')
    sim.dt = 0.286#0.05*P_K62b

    sim.add(m=Mstar_K62)
    #Don't do bottom sim.add, bc of below for loop
    #Take lenght of pl_param (pl_param) and retrieves information of each particle 
    for i in range(0,len(pl_param)):
        temp = pl_param[i]
        sim.add(m=temp[-1],a=temp[0],e=temp[1],inc=temp[2],omega=temp[3],Omega=temp[4],M=temp[5])
    sim.move_to_com()

    ps = sim.particles
       
    #F_t = np.zeros(len(days))
    fname = "Orb_data.txt"
    write2file("",'head',data_dir,fname) ##time,flux,ecc,varpi,inc,Omg\n
    K62f_idx = 5
    for d in range(0,len(days)):
        sim.integrate(days[d])
        ecc_t = ps[K62f_idx].e
        omg_t = np.degrees(ps[K62f_idx].omega) % 360.
        inc_t = np.degrees(ps[K62f_idx].inc)
        RA_t = np.degrees(ps[K62f_idx].Omega) % 360.
        MA_t = np.degrees(ps[K62f_idx].M) % 360.

        out_stg = "%1.6e %1.6f %1.6f %2.3f %2.3f %2.3f %2.3f\n" % (days[d]/365.25,ps[K62f_idx].a,ecc_t,inc_t,omg_t,RA_t,MA_t)
        write2file(out_stg,'foot',data_dir,fname)

    return 


run_sim()

0.285752535100395
